<a href="https://colab.research.google.com/github/ted-M-tech/data-science-1.3M-linkedin-jobs-skills/blob/makoto%2Fissue2/eda2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SET UP

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("asaniczka/1-3m-linkedin-jobs-and-skills-2024")

print("Path to dataset files:", path)

100%|██████████| 1.88G/1.88G [01:26<00:00, 23.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/asaniczka/1-3m-linkedin-jobs-and-skills-2024/versions/2


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import countDistinct, split, explode, trim, lower, regexp_replace, col, length, to_timestamp
import os
import pandas as pd

In [ ]:
# 1. Initialize the Session
spark = SparkSession.builder.appName("LinkedInAnalysis").getOrCreate()

# Construct the correct file paths
jobs_file_path = os.path.join(path, "linkedin_job_postings.csv")
skills_file_path = os.path.join(path, "job_skills.csv")
summary_file_path = os.path.join(path, "job_summary.csv")

In [ ]:
# 2. Load the datasets
    ## header=True to use the first row as column names
    ## inferSchema=True so Spark guesses if a column is a Number or a String

# Loading Linkedin Job Postings df
    ## Using multiline option as summaries include multiple paragraphs

df_jobs = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .option("multiline","true")\
  .load(jobs_file_path)\

# Cache df
df_jobs.cache()

# Sanity Check
print("Sanity Check: Total rows are matching - ", df_jobs.count() == 1348454) # matching total rows
print("Sanity Check: No unexpected nulls, with 10% margin of error - ", df_jobs.count() - df_jobs.na.drop().count() < (30*1.1) ) # checking for unexpected nulls, with 10% margin of error


Sanity Check: Total rows are matching -  True
Sanity Check: No unexpected nulls, with 10% margin of error -  True


In [ ]:
df_skills = spark.read.csv(skills_file_path, header=True, inferSchema=True)

In [ ]:
## Using multiline option as summaries include multiple paragraphs
## Escaping double quotes used in summaries

df_summary = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .option("multiline","true")\
  .option("quote", '"')\
  .option("escape", '\"')\
  .load(summary_file_path)

# Cache df
df_summary.cache()

# Sanity Checks
print("Sanity Check: Total rows are matching - ", df_summary.count() == 1297332) # matching total rows
print("Sanity Check: No unexpected nulls - ", df_summary.count() == df_summary.na.drop().count()) # checking for unexpected nulls

Sanity Check: Total rows are matching -  True
Sanity Check: No unexpected nulls -  True


# Display the Schema

In [ ]:
print("--- Jobs Schema ---")
df_jobs.printSchema()

print("--- Skills Schema ---")
df_skills.printSchema()

print("--- Summary Schema ---")
df_summary.printSchema()

--- Jobs Schema ---
root
 |-- job_link: string (nullable = true)
 |-- last_processed_time: string (nullable = true)
 |-- got_summary: string (nullable = true)
 |-- got_ner: string (nullable = true)
 |-- is_being_worked: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- company: string (nullable = true)
 |-- job_location: string (nullable = true)
 |-- first_seen: string (nullable = true)
 |-- search_city: string (nullable = true)
 |-- search_country: string (nullable = true)
 |-- search_position: string (nullable = true)
 |-- job_level: string (nullable = true)
 |-- job_type: string (nullable = true)

--- Skills Schema ---
root
 |-- job_link: string (nullable = true)
 |-- job_skills: string (nullable = true)

--- Summary Schema ---
root
 |-- job_link: string (nullable = true)
 |-- job_summary: string (nullable = true)



## Count Rows and Columns

In [ ]:
# Count Rows and Columns
print(f"Jobs Data: {df_jobs.count()} rows, {len(df_jobs.columns)} columns")
print(f"Skills Data: {df_skills.count()} rows, {len(df_skills.columns)} columns")
print(f"Summary Data: {df_summary.count()} rows, {len(df_summary.columns)} columns")

Jobs Data: 1348454 rows, 14 columns
Skills Data: 1296381 rows, 2 columns
Summary Data: 1297332 rows, 2 columns


## Descrive `df_jobs` Statistics

In [ ]:
#  Descriptive Statistics
print("--- Jobs Statistics ---")
df_jobs.describe().show()

--- Jobs Statistics ---
+-------+--------------------+-----------+-------+---------------+--------------------+-----------------+--------------------+-------------+-----------+--------------+----------------+--------------------+---------+
|summary|            job_link|got_summary|got_ner|is_being_worked|           job_title|          company|        job_location|   first_seen|search_city|search_country| search_position|           job_level| job_type|
+-------+--------------------+-----------+-------+---------------+--------------------+-----------------+--------------------+-------------+-----------+--------------+----------------+--------------------+---------+
|  count|             1348454|    1348454|1348454|        1348454|             1348454|          1348443|             1348435|      1348454|    1348454|       1348454|         1348454|             1348454|  1348453|
|   mean|                NULL|       NULL|   NULL|           NULL|                NULL|882.4166666666666|       

### Null Value Counts for `df_jobs`

In [ ]:
print("--- Null Counts for df_jobs ---")
for column in df_jobs.columns:
    null_count = df_jobs.filter(col(column).isNull()).count()
    print(f"Column '{column}': {null_count} nulls")

--- Null Counts for df_jobs ---
Column 'job_link': 0 nulls
Column 'last_processed_time': 0 nulls
Column 'got_summary': 0 nulls
Column 'got_ner': 0 nulls
Column 'is_being_worked': 0 nulls
Column 'job_title': 0 nulls
Column 'company': 11 nulls
Column 'job_location': 19 nulls
Column 'first_seen': 0 nulls
Column 'search_city': 0 nulls
Column 'search_country': 0 nulls
Column 'search_position': 0 nulls
Column 'job_level': 0 nulls
Column 'job_type': 1 nulls


### Null Value Counts for `df_skills`

In [ ]:
print("\n--- Null Counts for df_skills ---")
for column in df_skills.columns:
    null_count = df_skills.filter(col(column).isNull()).count()
    print(f"Column '{column}': {null_count} nulls")


--- Null Counts for df_skills ---
Column 'job_link': 0 nulls
Column 'job_skills': 2007 nulls


### Null Value Counts for `df_summary`

In [ ]:
print("\n--- Null Counts for df_summary ---")
for column in df_summary.columns:
    null_count = df_summary.filter(col(column).isNull()).count()
    print(f"Column '{column}': {null_count} nulls")


--- Null Counts for df_summary ---
Column 'job_link': 0 nulls
Column 'job_summary': 0 nulls


## Display the fiest 5 rows of each DataFrame

In [ ]:
print('--- Job Postings ---')
display(df_jobs.limit(5).toPandas())
print('\n--- Job Skills ---')
display(df_skills.limit(5).toPandas())
print('\n--- Job Summary ---')
display(df_summary.limit(5).toPandas())

--- Job Postings ---


,job_link,last_processed_time,got_summary,got_ner,is_being_worked,job_title,company,job_location,first_seen,search_city,search_country,search_position,job_level,job_type
0,https://www.linkedin.com/jobs/view/account-exe...,2024-01-21 07:12:29.002560,t,t,f,Account Executive - Dispensing (NorCal/Norther...,BD,"San Diego, CA",2024-01-15,Coronado,United States,Color Maker,Mid senior,Onsite
1,https://www.linkedin.com/jobs/view/registered-...,2024-01-21 07:39:58.881370,t,t,f,Registered Nurse - RN Care Manager,Trinity Health MI,"Norton Shores, MI",2024-01-14,Grand Haven,United States,Director Nursing Service,Mid senior,Onsite
2,https://www.linkedin.com/jobs/view/restaurant-...,2024-01-21 07:40:00.251126,t,t,f,RESTAURANT SUPERVISOR - THE FORKLIFT,Wasatch Adaptive Sports,"Sandy, UT",2024-01-14,Tooele,United States,Stand-In,Mid senior,Onsite
3,https://www.linkedin.com/jobs/view/independent...,2024-01-21 07:40:00.308133,t,t,f,Independent Real Estate Agent,Howard Hanna | Rand Realty,"Englewood Cliffs, NJ",2024-01-16,Pinehurst,United States,Real-Estate Clerk,Mid senior,Onsite
4,https://www.linkedin.com/jobs/view/group-unit-...,2024-01-19 09:45:09.215838,f,f,f,Group/Unit Supervisor (Systems Support Manager...,"IRS, Office of Chief Counsel","Chamblee, GA",2024-01-17,Gadsden,United States,Supervisor Travel-Information Center,Mid senior,Onsite



--- Job Skills ---


,job_link,job_skills
0,https://www.linkedin.com/jobs/view/housekeeper...,"Building Custodial Services, Cleaning, Janitor..."
1,https://www.linkedin.com/jobs/view/assistant-g...,"Customer service, Restaurant management, Food ..."
2,https://www.linkedin.com/jobs/view/school-base...,"Applied Behavior Analysis (ABA), Data analysis..."
3,https://www.linkedin.com/jobs/view/electrical-...,"Electrical Engineering, Project Controls, Sche..."
4,https://www.linkedin.com/jobs/view/electrical-...,"Electrical Assembly, Point to point wiring, St..."



--- Job Summary ---


,job_link,job_summary
0,https://www.linkedin.com/jobs/view/restaurant-...,Rock N Roll Sushi is hiring a Restaurant Manag...
1,https://www.linkedin.com/jobs/view/med-surg-re...,Schedule\n: PRN is required minimum 12 hours p...
2,https://www.linkedin.com/jobs/view/registered-...,Description\nIntroduction\nAre you looking for...
3,https://uk.linkedin.com/jobs/view/commercial-a...,Commercial account executive\nSheffield\nFull ...
4,https://www.linkedin.com/jobs/view/store-manag...,Address:\nUSA-CT-Newington-44 Fenn Road\nStore...


# CLEANING

Jobs Data Cleaning

In [ ]:
# 1. Clean Job Titles
    # We remove any character that is NOT a letter, number, space, or standard punctuation
df_jobs_clean = df_jobs.withColumn(
    "job_title_clean",
    regexp_replace(col("job_title"), r"[^a-zA-Z0-9\s\-\/\&]", "")
)

# 2. Clean Company Names
df_jobs_clean = df_jobs_clean.filter(
    (col("company").isNotNull())
)

# Changing Data Type to Boolean for columns "got_summary", "got_ner", and "is_being_worked".
# Additionally changing data type to timestamp for column "first_seen"
df_jobs_clean = df_jobs_clean.withColumn("got_summary", col("got_summary").cast(BooleanType())) \
            .withColumn("got_ner", col("got_ner").cast(BooleanType())) \
            .withColumn("is_being_worked", col("is_being_worked").cast(BooleanType())) \
            .withColumn("first_seen", to_timestamp(col("first_seen"), "yyyy-MM-dd"))

display(df_jobs_clean.limit(3).toPandas())

,job_link,last_processed_time,got_summary,got_ner,is_being_worked,job_title,company,job_location,first_seen,search_city,search_country,search_position,job_level,job_type,job_title_clean
0,https://www.linkedin.com/jobs/view/account-exe...,2024-01-21 07:12:29.002560,True,True,False,Account Executive - Dispensing (NorCal/Norther...,BD,"San Diego, CA",2024-01-15,Coronado,United States,Color Maker,Mid senior,Onsite,Account Executive - Dispensing NorCal/Northern...
1,https://www.linkedin.com/jobs/view/registered-...,2024-01-21 07:39:58.881370,True,True,False,Registered Nurse - RN Care Manager,Trinity Health MI,"Norton Shores, MI",2024-01-14,Grand Haven,United States,Director Nursing Service,Mid senior,Onsite,Registered Nurse - RN Care Manager
2,https://www.linkedin.com/jobs/view/restaurant-...,2024-01-21 07:40:00.251126,True,True,False,RESTAURANT SUPERVISOR - THE FORKLIFT,Wasatch Adaptive Sports,"Sandy, UT",2024-01-14,Tooele,United States,Stand-In,Mid senior,Onsite,RESTAURANT SUPERVISOR - THE FORKLIFT


Skills Data Cleaning

In [ ]:
# 1. Split the long string into an List based on commas
    # "Skill A, Skill B" -> ["Skill A", "Skill B"]
df_skills_array = df_skills.withColumn("skills_array", split(col("job_skills"), ","))

# 2. EXPLODE the array
    # This creates a new row for EVERY skill in the list.
df_skills_exploded = df_skills_array.select(
    col("job_link"),
    explode(col("skills_array")).alias("skill_raw")
)

# 3. Clean the individual skills
    # Remove leading spaces, convert to lowercase for consistency
df_skills_final = df_skills_exploded.withColumn("skill", trim(lower(col("skill_raw")))) \
                                    .filter(col("skill") != "") # Remove empty strings

# Check the difference
print(f"Original Skills Rows: {df_skills.count()}")
print(f"Deep Cleaned Skills Rows: {df_skills_final.count()}")
display(df_skills_final.limit(3).toPandas())

Original Skills Rows: 1296381
Deep Cleaned Skills Rows: 26908836


,job_link,skill_raw,skill
0,https://www.linkedin.com/jobs/view/housekeeper...,Building Custodial Services,building custodial services
1,https://www.linkedin.com/jobs/view/housekeeper...,Cleaning,cleaning
2,https://www.linkedin.com/jobs/view/housekeeper...,Janitorial Services,janitorial services


Summary Data Cleaning

In [ ]:
# Remove HTML tags using a regex pattern, then remove extra whitespace
df_summary_clean = df_summary.withColumn(
    "job_summary_clean",
    regexp_replace(col("job_summary"), r"<[^>]+>", "")
).withColumn(
    "job_summary_clean",
    regexp_replace(col("job_summary_clean"), r"\s+", " ")
)

display(df_summary_clean.limit(5).toPandas())

,job_link,job_summary,job_summary_clean
0,https://www.linkedin.com/jobs/view/restaurant-...,Rock N Roll Sushi is hiring a Restaurant Manag...,Rock N Roll Sushi is hiring a Restaurant Manag...
1,https://www.linkedin.com/jobs/view/med-surg-re...,Schedule\n: PRN is required minimum 12 hours p...,Schedule : PRN is required minimum 12 hours pe...
2,https://www.linkedin.com/jobs/view/registered-...,Description\nIntroduction\nAre you looking for...,Description Introduction Are you looking for a...
3,https://uk.linkedin.com/jobs/view/commercial-a...,Commercial account executive\nSheffield\nFull ...,Commercial account executive Sheffield Full ti...
4,https://www.linkedin.com/jobs/view/store-manag...,Address:\nUSA-CT-Newington-44 Fenn Road\nStore...,Address: USA-CT-Newington-44 Fenn Road Store C...


# Deeper analysis

In [ ]:
# Define which columns we want to check
columns_to_check = ['job_title', 'company', 'job_location', 'search_country', 'job_level']

print("--- Unique Value Counts (Cardinality) ---")

for column in columns_to_check:
    unique_count = df_jobs_clean.select(countDistinct(column)).collect()[0][0]
    print(f"Unique {column}s: {unique_count}")

# Check unique skills (from the skills dataframe)
unique_skills = df_skills.select(countDistinct("job_skills")).collect()[0][0]
print(f"\nUnique Skills found: {unique_skills}")

--- Unique Value Counts (Cardinality) ---
Unique job_titles: 584541
Unique companys: 90625
Unique job_locations: 29171
Unique search_countrys: 26
Unique job_levels: 23

Unique Skills found: 1287097


## Deep Dive into "Top Values"

In [ ]:
print("--- Top 10 Companies Posting Jobs ---")
df_jobs_clean.groupBy("company") \
    .count() \
    .orderBy(col("count").desc()) \
    .show(10, truncate=False)

print("\n--- Top 10 Job Titles ---")
df_jobs_clean.groupBy("job_title") \
    .count() \
    .orderBy(col("count").desc()) \
    .show(10, truncate=False)

print("\n--- Top 10 Skills Requested ---")
df_skills_final.groupBy("skill") \
    .count() \
    .orderBy(col("count").desc()) \
    .show(10, truncate=False)

--- Top 10 Companies Posting Jobs ---
+----------------------+-----+
|company               |count|
+----------------------+-----+
|Health eCareers       |41597|
|Jobs for Humanity     |27680|
|TravelNurseSource     |16142|
|Dollar General        |14815|
|PracticeLink          |9737 |
|Energy Jobline        |9364 |
|Gotham Enterprises Ltd|8935 |
|Jobot                 |8713 |
|ClearanceJobs         |8599 |
|McDonald's            |8125 |
+----------------------+-----+
only showing top 10 rows

--- Top 10 Job Titles ---
+-------------------------------+-----+
|job_title                      |count|
+-------------------------------+-----+
|LEAD SALES ASSOCIATE-FT        |7325 |
|Shift Manager                  |5818 |
|First Year Tax Professional    |5356 |
|Assistant Manager              |5346 |
|Customer Service Representative|5203 |
|LEAD SALES ASSOCIATE-PT        |4924 |
|Store Manager                  |4791 |
|CUSTOMER SERVICE REPRESENTATIVE|4218 |
|Registered Nurse               |419